In [1]:
import pandas as pd
from datasets import Dataset
import sys
sys.path.append('../src/')
from utils.data.prompt_utils import convert_prompstlye, tuluform, splitter
from utils.data.dataproc import indcomp

/u/prasanns/miniconda3/envs/fixenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
rjson = pd.read_json("../outputs/results/dynarmlogs/math/mathbigdata1b_1bjustoffpolicyrmupper.jsonl", orient='records', lines=True)

In [14]:
rjson

,texts,rewards,golds,call
0,[Question: (((3 * 7) + 1) + 8) = \n\nAnswer: ...,"[1.109375, 1.4609375]","[-14.0, -18.0]","[1, 1]"
1,[Question: (((3 - 7) * (9 + 2)) + 2) = \n\nAns...,"[1.6328125, 1.6328125]","[0.0, 0.0]","[1, 1]"
2,[Question: (((8 + 3) + 3) - 5) = \n\nAnswer: ...,"[1.3515625, 1.3125]","[-14.0, -14.0]","[1, 1]"
3,[Question: (((4 + 8) * 8) * 2) = \n\nAnswer: ...,"[1.328125, 1.1015625]","[0.0, -6.0]","[1, 1]"
4,[Question: (((4 - 1) * (6 + 2)) - ((5 - 3) * (...,"[1.0859375, 1.0234375]","[-21.0, -54.0]","[1, 1]"
...,...,...,...,...
63995,[Question: (((5 + 9) - 2) * 2) = \n\nAnswer: ...,"[1.453125, 0.2314453125]","[-22.0, -30.0]","[4000, 4000]"
63996,[Question: (((7 - 4) - (4 * 9)) * (8 + 8)) = \...,"[-1.0390625, 4.5625]","[-50.0, -47.0]","[4000, 4000]"
63997,[Question: (((9 * 3) * 1) - 2) = \n\nAnswer: ...,"[-0.220703125, 1.03125]","[-34.0, -23.0]","[4000, 4000]"
63998,[Question: (((4 + 7) + 6) * 6) = \n\nAnswer: ...,"[-3.5, 12.5625]","[-43.0, -3.0]","[4000, 4000]"


In [7]:
def acat(inp):
    a, b = splitter(inp)
    return a+" "+b

In [15]:
rjson['texts'] = [[acat(s) for s in r] for r in rjson['texts']]

In [16]:
rjson

,texts,rewards,golds,call
0,[(((3 * 7) + 1) + 8) = (21 + 8) = 29 = 34 = 34...,"[1.109375, 1.4609375]","[-14.0, -18.0]","[1, 1]"
1,[(((3 - 7) * (9 + 2)) + 2) = ((-4 * (9 + 2)) +...,"[1.6328125, 1.6328125]","[0.0, 0.0]","[1, 1]"
2,[(((8 + 3) + 3) - 5) = (11 + 3) = 14 = 17 = 17...,"[1.3515625, 1.3125]","[-14.0, -14.0]","[1, 1]"
3,[(((4 + 8) * 8) * 2) = ((12 * 8) * 2) = (96 * ...,"[1.328125, 1.1015625]","[0.0, -6.0]","[1, 1]"
4,[(((4 - 1) * (6 + 2)) - ((5 - 3) * (5 + 9))) =...,"[1.0859375, 1.0234375]","[-21.0, -54.0]","[1, 1]"
...,...,...,...,...
63995,[(((5 + 9) - 2) * 2) = (14 - 2) = 12 = 10 + 2\...,"[1.453125, 0.2314453125]","[-22.0, -30.0]","[4000, 4000]"
63996,[(((7 - 4) - (4 * 9)) * (8 + 8)) = (3 - (4 * 9...,"[-1.0390625, 4.5625]","[-50.0, -47.0]","[4000, 4000]"
63997,[(((9 * 3) * 1) - 2) = (27 * 1) = 27.\n27\n(27...,"[-0.220703125, 1.03125]","[-34.0, -23.0]","[4000, 4000]"
63998,[(((4 + 7) + 6) * 6) = (11 + 6) = 17.\n13\nWha...,"[-3.5, 12.5625]","[-43.0, -3.0]","[4000, 4000]"


In [17]:
def mroll_split(inp):
    ind = inp.index("=")+2
    return inp[:ind], inp[ind:]

In [18]:
def rollout_to_prefs(indf, ismath=True):
    prefdata = []
    sfunct = mroll_split if ismath else splitter
    for ind, row in indf.iterrows():
        for i in range(0, len(row['texts']), 2):
            
            q1, a1 = sfunct(row['texts'][i])
            q2, a2 = sfunct(row['texts'][i+1])
            assert q1==q2
            astrs = [a1, a2]
            indj, indk = indcomp(row['golds'][i:i+2], 0)
            tmp = {
                'question': q1,
                'response_j': astrs[indj],
                'response_k': astrs[indk],
                'score_j': row['golds'][i+indj], 
                'score_k': row['golds'][i+indk],
                'magnitude': row['golds'][indj] - row['golds'][indk]
            }
            prefdata.append(tmp)
            
    return Dataset.from_list(prefdata)

pdata = rollout_to_prefs(rjson)

In [21]:
pdata = pdata.shuffle(seed=0)

In [22]:
smallpdata = pdata.select(range(50000))

In [23]:
# smallpdata.save_to_disk("../outputs/data/eurusrm/offpolicydata1k")
# smallpdata.save_to_disk("../outputs/data/math/offp50k")

Saving the dataset (1/1 shards): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [00:00<00:00, 217838.17 examples/s]


In [25]:
# mprefs = Dataset.load_from_disk("../outputs/data/math/mathprefdata/")

In [26]:
# mprefs[0]

{'question': '(((3 * 4) - 2) - 1) = ',
 'response_j': '((12 - 2) - 1) = (13 - 1) = 8 = 9 = 8 = 9 = 9\n\n(13 - 1) = 8 = 7 = 7 = 7\n\n12 = 7 = 7\n\n12 = 7 = 7\n\n12 = 7\n\nM = 7\n\nM = 7\n\nM = 7\n\nM = 7\n\nM = 7\n\nM = 7\n\nM = 7\n\nM = 7',
 'response_k': '((12 - 2) - 1) = (9 - 1) = 8 = 7 = 7 = 7 = 7 = 7 = 7 = 7 = 7 = 7 = -7 = -7 = -7 = -7 = -7 = -7 = -63 = -63 = -63 = -63 = -63 = -63 = 539 = 493 = -549 = -5191 = -5491 = -5491 = -4971 = -4971 =',
 'score_j': 3.4722222222222223,
 'score_k': 3.194444444444444,
 'magnitude': 0.2777777777777781}